In [3]:
from pathlib import Path

from summary_abstractive.module_model_handler.ver2 import (
    FaiseqTranslationModelHandlerVer2WordEmbeddings, 
    TranslationResultContainer,
    EvaluationTargetTranslationPair)

ModuleNotFoundError: No module named 'summary_abstractive'